In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the AnimalShelter class
from animal_shelter import AnimalShelter

# Define the connection parameters
username = "aacuser"
password = "user"
host = "nv-desktop-services.apporto.com"
port = 31919
db = "AAC"
collection = "animals"

# Create an instance of AnimalShelter with the provided parameters
shelter = AnimalShelter(username, password, host, port, db, collection)

# Fetch data from the database
df = pd.DataFrame.from_records(shelter.read({}))

# Drop the '_id' column to avoid issues with ObjectId type
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Encode the logo image
image_filename = '/home/arnoldlimberg1_snhu/Desktop/Grazioso Salvare Logo.png'  # replace with your own image path
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Initialize the Dash app
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'10%', 'width':'10%'})),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        row_deletable=False,
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        data = shelter.read({})
    else:
        query = {}
        if filter_type == 'Water':
            query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}
        elif filter_type == 'Mountain':
            query = {"$and": [
                {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]}
        elif filter_type == 'Disaster':
            query = {"$and": [
                {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
            ]}
        data = shelter.read(query)
    
    # Drop the '_id' column to avoid issues with ObjectId type
    df = pd.DataFrame.from_records(data)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' not in dff.columns:
        return [html.Div("No data available for the selected filter.")]
    
    breed_counts = dff['breed'].value_counts()
    top_breeds = breed_counts.nlargest(10)
    other_breeds = breed_counts.iloc[10:].sum()
    top_breeds['Other'] = other_breeds
    
    top_breeds_df = top_breeds.reset_index()
    top_breeds_df.columns = ['breed', 'count']
    
    return [
        dcc.Graph(
            figure=px.pie(top_breeds_df, names='breed', values='count', title='Breed Distribution', 
                          height=400, width=400) 
        )
    ]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    center = [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=center, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run the app
app.run_server(port=8053, debug=True)